In [2]:
import os

TRAIN_SPLIT = 0.8

RAW_IMAGES = './hard_hat/images/'
RAW_ANNOTATIONS = './hard_hat/annotations/'
TRAIN_DIR = './data/train/'
TEST_DIR = './data/test/'

ANNOTATIONS_DIR = './annotations/'

In [3]:
# PRETRAINED_DIR = './pretrained_models/centernet_mobilenetv2_fpn_od/'
# MY_MODEL_DIR = './models/my_centernet/'

PRETRAINED_DIR = './pretrained_models/efficientdet_d0_coco17_tpu-32/'
MY_MODEL_DIR = './models/my_effiecientdet/'

In [4]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [5]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Images to train, test

In [4]:
# image_files = os.listdir(RAW_IMAGES)
# for idx, file in enumerate(image_files[:500]):
#     annot = file.split('.')[0] + '.xml'

#     try:
#         if idx < len(image_files[:500])*TRAIN_SPLIT:
#             !cp {RAW_ANNOTATIONS + annot} {TRAIN_DIR}
#             !cp {RAW_IMAGES + file} {TRAIN_DIR}
#         else:
#             !cp {RAW_ANNOTATIONS + annot} {TEST_DIR}
#             !cp {RAW_IMAGES + file} {TEST_DIR}
#     except:
#         print('{} ERROR!'.format(file))

## Label map and create .pbtxt

In [2]:
labels = [{'name': 'helmet', 'id': 1}, {'name': 'head', 'id': 2}, {'name': 'person', 'id': 3}]
labels

[{'name': 'helmet', 'id': 1},
 {'name': 'head', 'id': 2},
 {'name': 'person', 'id': 3}]

In [7]:
# with open(ANNOTATIONS_DIR + 'label_map.pbtxt', 'w') as f:
#     for label in labels:
#         f.write('item{\n')
#         f.write('\tname:\'{}\'\n'.format(label['name']))
#         f.write('\tid:{}\n'.format(label['id']))
#         f.write('}\n')

## .xml to .record

In [5]:
!python generate_tfrecord_from_xml.py -x {TRAIN_DIR} -i {TRAIN_DIR} -l {ANNOTATIONS_DIR + 'label_map.pbtxt'} -o {ANNOTATIONS_DIR + 'train.record'}
!python generate_tfrecord_from_xml.py -x {TEST_DIR} -i {TEST_DIR} -l {ANNOTATIONS_DIR + 'label_map.pbtxt'} -o {ANNOTATIONS_DIR + 'test.record'}

Successfully created the TFRecord file: ./annotations/train.record
Successfully created the TFRecord file: ./annotations/test.record


## Copy config to custom model dir

In [6]:
!mkdir {MY_MODEL_DIR}
!cp {PRETRAINED_DIR + 'pipeline.config'} {MY_MODEL_DIR}

mkdir: cannot create directory ‘./models/my_effiecientdet/’: File exists


## Update config for transfer learning

In [8]:
config = config_util.get_configs_from_pipeline_file(MY_MODEL_DIR + 'pipeline.config')
# config

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(MY_MODEL_DIR + 'pipeline.config', 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [10]:
pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_DIR + 'checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = ANNOTATIONS_DIR + 'label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATIONS_DIR + 'train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATIONS_DIR + 'label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATIONS_DIR + 'test.record']

In [11]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(MY_MODEL_DIR + 'pipeline.config', 'wb') as f:
    f.write(config_text)

## Train (terminal)

In [21]:
!echo python ../models/research/object_detection/model_main_tf2.py --model_dir={MY_MODEL_DIR} --pipeline_config_path={MY_MODEL_DIR + 'pipeline.config'} --num_train_steps=5000
# !echo python ../models/research/object_detection/model_main_tf2.py --model_dir={MY_MODEL_DIR} --pipeline_config_path={MY_MODEL_DIR + 'pipeline_adam.config'} --num_train_steps=5000

python ../models/research/object_detection/model_main_tf2.py --model_dir=./models/my_effiecientdet/ --pipeline_config_path=./models/my_effiecientdet/pipeline_adam.config --num_train_steps=5000


## Load model from ckpt

In [7]:
MY_MODEL_DIR = './models/my_mobilenet/'
CHECKPOINT = 'ckpt-8'

configs = config_util.get_configs_from_pipeline_file(MY_MODEL_DIR + 'pipeline.config')
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(MY_MODEL_DIR + CHECKPOINT).expect_partial()

2022-06-21 01:22:33.933492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 01:22:33.941670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 01:22:33.941900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 01:22:33.942658: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## Inference

In [9]:
category_index = label_map_util.create_category_index_from_labelmap('./annotations/label_map.pbtxt')
category_index

{1: {'id': 1, 'name': 'helmet'},
 2: {'id': 2, 'name': 'head'},
 3: {'id': 3, 'name': 'person'}}

In [12]:
image_np = np.array(cv2.imread(TEST_DIR + 'hard_hat_workers69.png'))
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

cv2.imshow('detection', image_np_with_detections)
cv2.waitKey(0)
cv2.destroyAllWindows()

# out_image = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
# plt.imshow(out_image)
# plt.show()

2022-06-21 01:21:11.930095: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2022-06-21 01:21:12.144648: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


## test video

In [23]:
cap = cv2.VideoCapture('./trim_hard_hat_demo_2.mp4')

frames = []
while cap.isOpened():
    ret, frame = cap.read()

    image_np = np.array(frame)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    print(ret)
    if ret:
        res = cv2.resize(image_np_with_detections, (854, 480))
        cv2.imshow('Demo', res)
        frames.append(res)

        if cv2.waitKey(15) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()

writer = cv2.VideoWriter('demo_detections.avi', cv2.VideoWriter_fourcc(*'mp4v'), 20, (854, 480))
for i in frames:
    writer.write(i)
writer.release()

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
